In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [2]:
# image_agent_with_approval.py
import asyncio
import uuid
import base64
from typing import List

# ADK / Google GenAI imports (from your resources)
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner, InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.tool_context import ToolContext

# MCP tool import you showed
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

# Retry config (from your sample)
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Create the MCP toolset pointing to the Everything Server and filtering to getTinyImage
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=["-y", "@modelcontextprotocol/server-everything"],
            # filter to only expose the tiny image helper we care about
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created (mcp_image_server)")

# ---------- Long-running pausable tool: generate_images ----------
# Logic:
# - If num_images == 1: auto-approve, call MCP once and return image(s)
# - If num_images > 1 and this is first call (no tool_context.tool_confirmation): request_confirmation() and return pending
# - If num_images > 1 and tool_context.tool_confirmation exists: check confirmed, then if approved call MCP sequentially num_images times and return list

def generate_images(num_images: int, prompt: str, tool_context: ToolContext) -> dict:
    """
    Tool that generates tiny images using mcp_image_server. Requires human approval
    for bulk generation (num_images > 1). Uses ToolContext to request confirmation.
    Returns a dict with status and either images or a pending/rejected message.

    NOTE: Replace the example MCP call sections below with the specific call your
    McpToolset provides in your ADK version. I provide plausible examples; adapt as needed.
    """
    # Basic validation
    if num_images <= 0:
        return {"status": "error", "message": "num_images must be >= 1"}

    # Fast path: single image -> auto-approve and generate immediately
    if num_images == 1:
        # --- MCP CALL HERE (single image) ---
        # Example: if mcp_image_server exposes a method `call_tool` or `run_tool`:
        # result = mcp_image_server.call_tool("getTinyImage", {"prompt": prompt})
        # Or if it is a callable object:
        # result = mcp_image_server.run({"tool": "getTinyImage", "prompt": prompt})
        #
        # For the example, we'll simulate getting back a base64 PNG returned as `image_b64`.
        # Replace the simulation below with the real mcp call.
        #
        # --- EXAMPLE (placeholder) ---
        try:
            # Attempt a plausible call pattern (you may need to change this):
            # result = mcp_image_server.generate(prompt=prompt)  # <-- adapt
            # image_b64 = result.get("image_b64")
            # For now: simulate
            image_b64 = "SIMULATED_BASE64_IMAGE_SINGLE"
            return {
                "status": "approved",
                "num_images": 1,
                "images_b64": [image_b64],
                "message": f"Generated 1 image for prompt: {prompt}",
            }
        except Exception as e:
            return {"status": "error", "message": f"MCP error: {e}"}

    # Bulk path: num_images > 1
    # Scenario 2: first call -> ask for confirmation and return pending
    LARGE_BULK_THRESHOLD = 1  # for our behavior, bulk = >1
    if num_images > LARGE_BULK_THRESHOLD and not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ You requested {num_images} images for prompt: '{prompt}'. Approve bulk generation?",
            payload={"num_images": num_images, "prompt": prompt},
        )
        return {
            "status": "pending",
            "message": f"Request for {num_images} images requires approval",
        }

    # Scenario 3: resumed call, we have a tool_confirmation
    if tool_context.tool_confirmation:
        if tool_context.tool_confirmation.confirmed:
            # Human approved -> call MCP sequentially num_images times
            images = []
            errors = []
            for i in range(num_images):
                try:
                    # --- MCP CALL HERE (sequential) ---
                    # Replace the simulation below with your actual MCP invocation.
                    #
                    # Example plausible forms:
                    # result = mcp_image_server.call_tool("getTinyImage", {"prompt": prompt})
                    # image_b64 = result.get("image_b64")
                    #
                    # or (if the tool exposes a 'run' coroutine):
                    # result = await mcp_image_server.run_tool("getTinyImage", {"prompt": prompt})
                    # image_b64 = result["image_b64"]
                    #
                    # For this example code (synchronous function expected by FunctionTool),
                    # assume a synchronous call exists. Adapt if yours uses async.
                    #
                    image_b64 = f"SIMULATED_BASE64_IMAGE_{i+1}"
                    images.append(image_b64)
                except Exception as e:
                    errors.append(str(e))
            if errors:
                return {
                    "status": "partial",
                    "num_images_requested": num_images,
                    "images_b64": images,
                    "errors": errors,
                    "message": f"Generated {len(images)} images; {len(errors)} errors",
                }
            else:
                return {
                    "status": "approved",
                    "num_images": num_images,
                    "images_b64": images,
                    "message": f"Generated {num_images} images for prompt: {prompt}",
                }
        else:
            # Human rejected
            return {
                "status": "rejected",
                "message": f"Bulk image generation of {num_images} images rejected by human",
            }

    # Fallback (shouldn't hit)
    return {"status": "error", "message": "Unhandled state in generate_images tool"}


print("✅ Pausable generate_images tool defined")


# ---------- Create the image agent and wrap in a resumable App ----------
image_agent = LlmAgent(
    name="image_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are an image generation assistant.

When asked to generate images:
1) Use the generate_images tool with num_images and prompt.
2) If the tool returns status 'pending', tell the user approval is required.
3) After approval/resume returns, present a concise summary with:
   - status (approved/rejected)
   - number of images
   - (if approved) indicate images were generated (we return them as base64)
4) Keep responses concise and informative.
""",
    # add the pausable tool into the agent's toolset
    tools=[FunctionTool(func=generate_images), mcp_image_server],
)

print("✅ Image LlmAgent created (with generate_images & MCP tool available)")

image_app = App(
    name="image_generator_app",
    root_agent=image_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)
print("✅ Resumable App created")

# Session service and runner
session_service = InMemorySessionService()
image_runner = Runner(app=image_app, session_service=session_service)
print("✅ Runner created")

# ---------- Helper functions for approval handling & printing ----------
def check_for_approval(events) -> dict:
    """Inspect events for adk_request_confirmation function call and return approval info or None."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.function_call and part.function_call.name == "adk_request_confirmation":
                    return {"approval_id": part.function_call.id, "invocation_id": event.invocation_id}
    return None

def print_agent_response(events):
    """Print agent text parts from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

def create_approval_response(approval_info: dict, approved: bool) -> types.Content:
    """Wrap the human confirmation decision into an ADK FunctionResponse Content object."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(role="user", parts=[types.Part(function_response=confirmation_response)])


# ---------- The workflow: runs the agent, detects approval events, resumes with decision ----------
async def run_image_workflow(query: str, num_images: int, auto_approve: bool = True):
    """
    Runs an image generation request through the resumable app and handles approvals.

    Args:
        query: prompt for the image(s)
        num_images: how many images to generate
        auto_approve: when approval is required, automatically choose approved/rejected
                      (useful for demo automation)
    """
    print("\n" + "=" * 70)
    print(f"User > Generate {num_images} image(s) for prompt: {query}\n")

    session_id = f"img_{uuid.uuid4().hex[:8]}"
    await session_service.create_session(
        app_name="image_generator_app", user_id="demo_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=f"Generate {num_images} images: {query}")])
    events = []

    # STEP 1: initial run
    async for event in image_runner.run_async(user_id="demo_user", session_id=session_id, new_message=query_content):
        events.append(event)

    # Check agent messages (these may include the tool call function parts)
    print_agent_response(events)

    # STEP 2: detect if an approval event was emitted
    approval_info = check_for_approval(events)
    if approval_info:
        print("⏸️  Paused for approval.")
        print(f"🤔 Human Decision (simulated): {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # STEP 3: resume execution by sending the confirmation response and same invocation_id
        reply_content = create_approval_response(approval_info, auto_approve)

        async for event in image_runner.run_async(
            user_id="demo_user",
            session_id=session_id,
            new_message=reply_content,
            invocation_id=approval_info["invocation_id"],  # critical to resume
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

    # If no approval was needed, we already printed the agent response above
    print("=" * 70 + "\n")


# ---------- Demo runner ----------
async def demo():
    # Demo 1: single image (auto-approved)
    await run_image_workflow("A small yellow banana on a white background", num_images=1, auto_approve=True)

    # Demo 2: bulk images with simulated APPROVE
    await run_image_workflow("A dramatic mountain landscape at sunrise", num_images=5, auto_approve=True)

    # Demo 3: bulk images with simulated REJECT
    await run_image_workflow("A herd of unicorns dancing in a meadow", num_images=4, auto_approve=False)

if __name__ == "__main__":
    await demo()



✅ MCP Tool created (mcp_image_server)
✅ Pausable generate_images tool defined
✅ Image LlmAgent created (with generate_images & MCP tool available)
✅ Resumable App created
✅ Runner created

User > Generate 1 image(s) for prompt: A small yellow banana on a white background



/tmp/ipykernel_13/4074667939.py:184: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),
/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


Agent > The image has been generated.


User > Generate 5 image(s) for prompt: A dramatic mountain landscape at sunrise



/usr/local/lib/python3.11/dist-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
/usr/local/lib/python3.11/dist-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Paused for approval.
🤔 Human Decision (simulated): APPROVE ✅

Agent > The 5 images have been generated.


User > Generate 4 image(s) for prompt: A herd of unicorns dancing in a meadow



⏸️  Paused for approval.
🤔 Human Decision (simulated): REJECT ❌

Agent > The image generation request for 4 images was rejected.

